In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 경고 메시지가 뜨지 않게 설정
import warnings 
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 인코딩
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# 검정
from scipy import stats
# 로지스틱 모델 만들기
from statsmodels.formula.api import logit

# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [2]:
df = pd.read_parquet('open/concat/2018_성과정보.parquet')

In [3]:
# 데이터의 도수의 종류가 1개인 데이터를 변수에 담는다.
cols_same_value = df.columns[df.nunique() == 1]

In [4]:
cols_same_value

Index([], dtype='object')

In [5]:
df

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.000000,0.0,0.409766,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,201807,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.000000,0.0,-1.010817,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,201807,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.000000,0.0,-0.902166,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,201807,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.000000,0.0,-0.866052,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,201807,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,201812,TEST_99995,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,-0.000026,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,NaN,NaN
2999996,201812,TEST_99996,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.300200,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.143554,0.233616,0.000000,8.564683,11.379632
2999997,201812,TEST_99997,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000049,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2999998,201812,TEST_99998,-0.048050,-0.048050,-0.048050,0.000000,0.0,-0.166447,0.0,0.196886,...,0.999998,0.333183,0.999998,0.999998,0.999998,-0.038153,-0.106142,0.000000,-0.241530,0.499079


In [6]:
df.select_dtypes(include = ['object', 'category'])

,ID
0,TRAIN_000000
1,TRAIN_000001
2,TRAIN_000002
3,TRAIN_000003
4,TRAIN_000004
...,...
2999995,TEST_99995
2999996,TEST_99996
2999997,TEST_99997
2999998,TEST_99998


In [8]:
na_box = []

for idx, val in df.isna().sum().items() :
    # print(f'{idx}의 결측치 : {val}')
    if val > 0 :
        na_box.append(idx)

In [11]:
df[na_box].isna().sum()/df.shape[0]

혜택수혜율_R3M    0.204154
혜택수혜율_B0M    0.232106
dtype: float64

In [13]:
# 혜택수혜율_R3M
print(df['혜택수혜율_R3M'].mean())
print(df['혜택수혜율_R3M'].median())

4.778079812012009
0.76201557


In [14]:
# 혜택수혜율_B0M
print(df['혜택수혜율_B0M'].mean())
print(df['혜택수혜율_B0M'].median())

5.554676373753143
0.98686008


- 두 칼럼 모두 평균과 중앙 값의 차이가 크기 때문에 중앙값으로 대체한다.

In [15]:
medi1 = df['혜택수혜율_R3M'].median()
medi2 = df['혜택수혜율_B0M'].median()

df.fillna({'혜택수혜율_R3M' : medi1, '혜택수혜율_B0M' : medi2}, inplace = True)

In [16]:
df.isna().sum().sum()

0

In [17]:
df.to_csv('data/성과정보_전처리.csv', index = False)